## combining chromosome SNP data per sample into WES SNP data
### also add EB data if available to filter out bad SNPs

In [1]:
# HOME
home = '/Users/mahtin'
home = '/Users/martinscience'
# standard paths
testdata = os.path.join(home,"Dropbox/Icke/Work/somVar/testdata")
static_path = os.path.join(home, "Dropbox/Icke/Work/static")
cluster_path = os.path.join(home, "/Users/martinscience/mount")
tooldata = os.path.join(home, "Dropbox/Icke/Work/somVar/tooldata")

# tool-specific paths
shell_path = "../shell"
# 
cnvdata = os.path.join(tooldata, "myCNVdata")
output_path = os.path.join(cnvdata, "output")

# the path to the input data
cnv_path = os.path.join(cnvdata, "cnv")
cnvPON_path = cnv_path = os.path.join(cnvdata, "chromCov")
cnv_path = os.path.join(cluster_path, "scratch/develop/PONcoverage/cnv")
cnvPON_path = os.path.join(cluster_path, "scratch/develop/PONcoverage/chromCov")

In [2]:
cnvPON_path

'/Users/martinscience/mount/scratch/develop/PONcoverage/chromCov'

In [3]:
test = pd.read_csv(f'{cnv_path}/01_A.chr3.snpEB', sep='\t').loc[:,['Chr', 'Start', 'Ref','EBscore', 'PoN-Alt']]
test[:13]
test = pd.read_csv(f'{cnv_path}/01_A.chr3.snp', sep='\t')
test[:13]

,Chr,Start,ExonPos,Ref,Depth,Alt,VAF
0,chr3,197596,50,T,112,C1,0.008929
1,chr3,197682,136,T,197,G2,0.010152
2,chr3,328224,768,A,188,T1,0.005319
3,chr3,340795,950,T,348,A1,0.002874
4,chr3,340862,1017,C,536,G1,0.001866
5,chr3,342011,1209,G,160,A1,0.006250
6,chr3,342964,1360,C,179,A1,0.005587
7,chr3,343057,1453,G,236,T1,0.004237
8,chr3,344564,1484,C,119,T60,0.504202
9,chr3,344607,1527,G,192,T1,0.005208


### combine all snp for a sample into one df with corresponding EBscores

In [4]:
# try with pd merge
import os
chrom_list = [f"chr{chrom + 1}" for chrom in range(22)] + ['chrX']


def combine_SNPdata(sample, cnv_path):
    snp_dfs = []
    file_base = os.path.join(cnv_path, sample)
    for chrom in chrom_list:
        # reading SNP
        file = f"{file_base}.{chrom}.snp"
        if not os.path.isfile(file):
            print('No file', file)
            continue
        print(f"Reading SNP VAF from {chrom} of sample {sample} from {file}.")
        snp_df = pd.read_csv(file, sep='\t')
        snp_df[['Alt', 'AltDepth']] = snp_df['Alt'].str.extract(r"([AGCT])([0-9]+)")
        
        #reading snpEB
        file = f"{file_base}.{chrom}.snpEB"
        if not os.path.isfile(file):
            print('No file', file)
            continue
        print(f"Reading SNP EBscores from {chrom} of sample {sample} from {file}.")
        snpEB_df = pd.read_csv(file, sep='\t').loc[:,['Chr', 'Start', 'Ref','Alt', 'EBscore', 'PoN-Alt']]
        snp_df = snp_df.merge(snpEB_df, on=['Chr', 'Start', 'Ref', 'Alt'])
        
        snp_dfs.append(snp_df)
    snp_df = pd.concat(snp_dfs)
    return snp_df.loc[:, ["Chr", "Start", "ExonPos", "Ref", "Depth", "Alt", "VAF", "EBscore", "PoN-Alt"]]

In [207]:
sample = "01_A"
snp_df = combine_SNPdata(sample, cnv_path)
snp_df

Reading SNP VAF from chr1 of sample 01_A from /Users/martinscience/mount/scratch/develop/PONcoverage/cnv/01_A.chr1.snp.
Reading SNP EBscores from chr1 of sample 01_A from /Users/martinscience/mount/scratch/develop/PONcoverage/cnv/01_A.chr1.snpEB.
Reading SNP VAF from chr2 of sample 01_A from /Users/martinscience/mount/scratch/develop/PONcoverage/cnv/01_A.chr2.snp.
Reading SNP EBscores from chr2 of sample 01_A from /Users/martinscience/mount/scratch/develop/PONcoverage/cnv/01_A.chr2.snpEB.
Reading SNP VAF from chr3 of sample 01_A from /Users/martinscience/mount/scratch/develop/PONcoverage/cnv/01_A.chr3.snp.
Reading SNP EBscores from chr3 of sample 01_A from /Users/martinscience/mount/scratch/develop/PONcoverage/cnv/01_A.chr3.snpEB.
Reading SNP VAF from chr4 of sample 01_A from /Users/martinscience/mount/scratch/develop/PONcoverage/cnv/01_A.chr4.snp.
Reading SNP EBscores from chr4 of sample 01_A from /Users/martinscience/mount/scratch/develop/PONcoverage/cnv/01_A.chr4.snpEB.
Reading SNP 

,Chr,Start,ExonPos,Ref,Depth,Alt,VAF,EBscore,PoN-Alt
0,chr1,14930,836,A,64,G,0.046875,0.505,1|2|1|1|3|0|2|2|0|0|1|0|0|0|0|0|1|0|2-0|2|0|0|...
1,chr1,15820,991,g,13,T,0.846154,2.401,16|0|0|1|0|0|28|0|0|0|0|0|0|0|0|0|3|0|0-0|0|0|...
2,chr1,16949,1410,A,182,C,0.038462,0.006,10|27|7|23|12|4|8|3|2|0|0|1|4|1|1|0|3|1|4-0|53...
3,chr1,17365,1689,C,285,G,0.140351,0.666,26|0|0|0|17|20|0|0|0|16|6|13|3|14|9|1|0|0|11-1...
4,chr1,17375,1699,A,353,G,0.065156,1.526,15|0|1|0|0|0|0|0|0|0|0|0|3|3|0|0|0|0|0-0|0|0|0...
...,...,...,...,...,...,...,...,...,...
9082,chrX,156023260,1835187,G,25,C,0.880000,0.950,4|6|1|5|7|9|5|3|4|2|3|1|2|3|6|1|3|3|4-25|20|31...
9083,chrX,156023266,1835193,C,25,T,0.880000,0.930,3|6|1|3|5|7|3|3|2|2|2|1|2|3|5|0|3|3|3-25|16|29...
9084,chrX,156024203,1835423,G,53,A,0.132075,0.392,2|2|0|13|2|0|4|6|0|1|0|0|0|0|4|2|2|2|5-3|2|0|3...
9085,chrX,156024239,1835459,C,56,G,0.267857,0.203,15|9|5|18|8|3|5|8|6|6|0|8|1|5|10|9|4|5|5-6|3|3...


## merge sample coverage with Pon coverage

### make full ExomCoords
+ get the last coords of the chromosome
+ make a running sum
+ add that to the ExonPos

In [86]:
def get_full_exon_pos(df):
    '''
    adds the accumulated exonic position (over all chroms)
    '''
    
    # save the output columns
    cols = list(df.columns)
    df = df.reset_index(drop=True)
    # adds the last ExonPos of chrom to start of next chromosome
    df.loc[:,'chromStep'] = df.shift(1)['ExonPos'].fillna(0).astype(int)
    df.loc[df['Chr'] == df.shift(1)['Chr'],'chromStep'] = 0
    df['chromAccum'] = df['chromStep'].cumsum()
    df['FullExonPos'] = df['ExonPos'] + df['chromAccum']
    cols = cols[:2] + ['FullExonPos'] + cols[2:] + ['chromAccum']
    return df[cols]


def combine_Covdata(sample, sample_cnv_path="", PON_cnv_path=""):
    
    cover_dfs = []

    for chrom in chrom_list:
        # reading sampleCoverage
        sample_cov_file = os.path.join(sample_cnv_path, f"{sample}.{chrom}.cov")
        if not os.path.isfile(sample_cov_file):
            print("No file", sample_cov_file)
            continue
        print(f"Reading coverage from {chrom} of sample {sample} from {sample_cov_file}.")
        cov_df = pd.read_csv(sample_cov_file, sep='\t', compression="gzip")
        
        #reading PONcoverage
        pon_cov_file = os.path.join(PON_cnv_path, f"{chrom}.filtered.csv.gz")
        if not os.path.isfile(pon_cov_file):
            print("No file", pon_cov_file)
            continue
        print(f"Reading PON coverage of {chrom} from file {pon_cov_file}.")
        pon_df = pd.read_csv(pon_cov_file, sep='\t', compression="gzip").loc[:,['Chr', 'Pos', 'ExonPos', 'meanCov', 'medianCov', 'std']]
        # column rename
        trans_dict = {col:f"PON{col}" for col in pon_df.columns[3:]}
        pon_df = pon_df.rename(columns=trans_dict)
        # merge sample with PON coverage
        sample_df = cov_df.merge(pon_df, on=['Chr', 'Pos', 'ExonPos'], how="inner").loc[:,['Chr', 'Pos', 'ExonPos', 'Coverage','PONmeanCov', 'PONmedianCov', 'PONstd']]
        
        # normalize the coverage
        sample_df['Coverage'] = sample_df['Coverage'] / sample_df['Coverage'].mean() * 100
        cover_dfs.append(sample_df)   
    cover_df = pd.concat(cover_dfs)
    cover_df['log2ratio'] = np.log2(cover_df['Coverage'] / cover_df['PONmeanCov'])    
    return get_full_exon_pos(cover_df)

In [87]:
sample = "01_A"
cov_df = combine_Covdata(sample, sample_cnv_path=cnv_path, PON_cnv_path=cnvPON_path)
cov_df

Reading coverage from chr1 of sample 01_A from /Users/martinscience/mount/scratch/develop/PONcoverage/cnv/01_A.chr1.cov.
Reading PON coverage of chr1 from file /Users/martinscience/mount/scratch/develop/PONcoverage/chromCov/chr1.filtered.csv.gz.
Reading coverage from chr2 of sample 01_A from /Users/martinscience/mount/scratch/develop/PONcoverage/cnv/01_A.chr2.cov.
Reading PON coverage of chr2 from file /Users/martinscience/mount/scratch/develop/PONcoverage/chromCov/chr2.filtered.csv.gz.
Reading coverage from chr3 of sample 01_A from /Users/martinscience/mount/scratch/develop/PONcoverage/cnv/01_A.chr3.cov.
Reading PON coverage of chr3 from file /Users/martinscience/mount/scratch/develop/PONcoverage/chromCov/chr3.filtered.csv.gz.
Reading coverage from chr4 of sample 01_A from /Users/martinscience/mount/scratch/develop/PONcoverage/cnv/01_A.chr4.cov.
Reading PON coverage of chr4 from file /Users/martinscience/mount/scratch/develop/PONcoverage/chromCov/chr4.filtered.csv.gz.
Reading coverage

,Chr,Pos,FullExonPos,ExonPos,Coverage,PONmeanCov,PONmedianCov,PONstd,log2ratio,chromAccum
0,chr1,16750,1211,1211,73.635666,45.918297,45.918297,18.882631,0.681336,0
1,chr1,16950,1411,1411,71.948120,31.275524,30.173918,13.179594,1.201923,0
2,chr1,17000,1461,1461,59.263895,21.266547,19.518825,10.704707,1.478568,0
3,chr1,17300,1624,1624,65.159679,19.347489,19.595966,6.948574,1.751833,0
4,chr1,17350,1674,1674,106.942382,38.241357,38.241357,11.840340,1.483628,0
...,...,...,...,...,...,...,...,...,...,...
690681,chrX,156024250,49447809,1835470,24.707203,22.130863,22.130863,9.875716,0.158872,47612339
690682,chrX,156025350,49448172,1835833,25.119488,22.331925,20.457632,12.148273,0.169699,47612339
690683,chrX,156027300,49448383,1836044,23.984212,28.676397,30.138524,11.504929,-0.257779,47612339
690684,chrX,156027350,49448433,1836094,22.850927,29.436317,29.042386,13.134193,-0.365345,47612339


### get the FullExonPos for snp_df from cov_df

In [208]:
def get_full_exon_pos_from_cov(snp_df, cov_df):
    snp_cols = list(snp_df.columns)
    snp_df = snp_df.merge(cov_df.loc[:,['Chr', 'chromAccum']].groupby('Chr').first().reset_index(), on='Chr')
    snp_df['FullExonPos'] = snp_df['ExonPos'] + snp_df['chromAccum']
    cols = snp_cols[:2] + ['FullExonPos'] + snp_cols[2:]
    return snp_df[cols]

snp_df = get_full_exon_pos_from_cov(snp_df, cov_df)
snp_df

,Chr,Start,FullExonPos,ExonPos,Ref,Depth,Alt,VAF,EBscore,PoN-Alt
0,chr1,14930,836,836,A,64,G,0.046875,0.505,1|2|1|1|3|0|2|2|0|0|1|0|0|0|0|0|1|0|2-0|2|0|0|...
1,chr1,15820,991,991,g,13,T,0.846154,2.401,16|0|0|1|0|0|28|0|0|0|0|0|0|0|0|0|3|0|0-0|0|0|...
2,chr1,16949,1410,1410,A,182,C,0.038462,0.006,10|27|7|23|12|4|8|3|2|0|0|1|4|1|1|0|3|1|4-0|53...
3,chr1,17365,1689,1689,C,285,G,0.140351,0.666,26|0|0|0|17|20|0|0|0|16|6|13|3|14|9|1|0|0|11-1...
4,chr1,17375,1699,1699,A,353,G,0.065156,1.526,15|0|1|0|0|0|0|0|0|0|0|0|3|3|0|0|0|0|0-0|0|0|0...
...,...,...,...,...,...,...,...,...,...,...
336263,chrX,156023260,49447526,1835187,G,25,C,0.880000,0.950,4|6|1|5|7|9|5|3|4|2|3|1|2|3|6|1|3|3|4-25|20|31...
336264,chrX,156023266,49447532,1835193,C,25,T,0.880000,0.930,3|6|1|3|5|7|3|3|2|2|2|1|2|3|5|0|3|3|3-25|16|29...
336265,chrX,156024203,49447762,1835423,G,53,A,0.132075,0.392,2|2|0|13|2|0|4|6|0|1|0|0|0|0|4|2|2|2|5-3|2|0|3...
336266,chrX,156024239,49447798,1835459,C,56,G,0.267857,0.203,15|9|5|18|8|3|5|8|6|6|0|8|1|5|10|9|4|5|5-6|3|3...


### rolling window std diff

In [359]:
def rolling_SNP_metrix(snp_df, chrom='',window_size=20, center=0.5):
    '''
    creates metrices to define heteroSNP gaps
    '''
    
    # reduce to hetero_SNPs
    df = snp_df.query('Chr == @chrom')
    # compute the heteroSNP center as the mean of all VAFs

    df.loc[:,'offCenter'] = np.abs(df['VAF'] - center)
    df.loc[:,'stdL'] = df.rolling(window_size)['VAF'].std()
    df.loc[:,'stdR'] = df.shift(-window_size + 1)['stdL']
    df.loc[:,'sumL'] = df.rolling(window_size)['offCenter'].sum()
    df.loc[:,'sumR'] = df.shift(-window_size + 1)['sumL']
    df.loc[:,'dVAFL'] = df.rolling(window_size)['deltaVAF'].sum()
    df.loc[:,'dVAFR'] = df.shift(-window_size + 1)['dVAFL']
    # fill up the margins
    for col in ['stdL', 'sumL', 'dVAFL']:
        df[col] = df[col].fillna(method='bfill')
    for col in ['stdR', 'sumR', 'dVAFR']:
        df[col] = df[col].fillna(method='ffill')
    df.loc[:,'scoreL'] = df['sumL'] + df['dVAFL']
    df.loc[:,'scoreR'] = df['sumR'] + df['dVAFL']
    return df

In [363]:
def mark_heteroSNP(snp_df, config={}):
    '''
    creates the heteroSNP metrices for the entire exome
    '''
    # reduce the snp_df
    VAFmin = config['VAFlimits'][0]
    VAFmax = config['VAFlimits'][1]
    minDepth = config['minDepth']
    minEBscore = config['minEBscore']
    df = snp_df.query('@VAFmin < VAF < @VAFmax').query('Depth >= @minDepth and EBscore > @minEBscore')
    # find the center
    center = df['VAF'].mean()
    print("heteroSNP centered around", center)
    # get the local VAF difference
    df['deltaVAF'] = np.abs(df['VAF'] - df.shift(1)['VAF']).fillna(0)
    
    # get the 
    std_dfs = []
    for chrom in df['Chr'].unique():
        std_df = rolling_SNP_metrix(df, chrom=chrom, window_size=config['windowSize'], center=center)
        std_dfs.append(std_df)
    std_df = pd.concat(std_dfs)
    
    # normalize the values globally
    std_df.loc[:,'scoreL'] = std_df['scoreL'] / std_df['scoreL'].max()
    std_df.loc[:,'scoreR'] = std_df['scoreR'] / std_df['scoreR'].max()
    # maximum of scoreDiff marks the breakpoints
    std_df.loc[:,'scoreDiff'] = std_df['scoreR'] - std_df['scoreL']
    std_df.loc[:,'scoreDiff'] = (std_df['scoreDiff'] / std_df['scoreDiff'].max() / 2) + center
    return std_df

In [364]:
config = {
    'VAFlimits':[0.05,0.95],
    'windowSize': 50,
    'minEBscore':0.5,
    'minDepth': 30
    
}
std_df = mark_heteroSNP(snp_df, config=config)
std_df

heteroSNP centered around 0.45567272445168566


/Users/martinscience/miniconda/envs/HDR2-env/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/martinscience/miniconda/envs/HDR2-env/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
<ipython-input-359-c0a60f0efc42>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

,Chr,Start,FullExonPos,ExonPos,Ref,Depth,Alt,VAF,EBscore,PoN-Alt,...,offCenter,stdL,stdR,sumL,sumR,dVAFL,dVAFR,scoreL,scoreR,scoreDiff
3,chr1,17365,1689,1689,C,285,G,0.140351,0.666,26|0|0|0|17|20|0|0|0|16|6|13|3|14|9|1|0|0|11-1...,...,0.315322,0.144470,0.144470,4.318947,4.318947,4.979714,4.979714,0.265615,0.272291,0.463029
4,chr1,17375,1699,1699,A,353,G,0.065156,1.526,15|0|1|0|0|0|0|0|0|0|0|0|3|3|0|0|0|0|0-0|0|0|0...,...,0.390517,0.144470,0.138932,4.318947,4.068822,4.979714,5.023077,0.265615,0.264967,0.454959
5,chr1,17379,1703,1703,G,363,A,0.093664,1.578,20|0|22|0|0|0|0|0|0|0|0|0|0|0|0|0|3|0|0-3|0|5|...,...,0.362009,0.144470,0.129318,4.318947,3.709589,4.979714,5.044362,0.265615,0.254448,0.443369
8,chr1,17614,1938,1938,G,216,A,0.171296,1.016,9|1|0|0|3|3|0|0|0|3|1|2|0|0|4|0|0|0|3-17|0|0|0...,...,0.284377,0.144470,0.119883,4.318947,3.382963,4.979714,5.082521,0.265615,0.244883,0.432831
12,chr1,69968,3629,3629,A,32,G,0.093750,1.149,0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0-0|0|0|0|...,...,0.361923,0.144470,0.113784,4.318947,3.130134,4.979714,5.071819,0.265615,0.237480,0.424674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336251,chrX,156010159,49446143,1833804,A,59,G,0.576271,0.927,13|17|16|0|0|0|0|20|0|0|0|21|0|6|0|0|0|19|2-4|...,...,0.120598,0.080648,0.120753,2.312866,3.335955,2.944844,3.589338,0.150186,0.183920,0.492840
336252,chrX,156010162,49446146,1833807,A,58,G,0.586207,1.072,12|18|16|0|0|0|0|21|0|0|0|20|0|6|0|0|0|19|0-5|...,...,0.130534,0.082125,0.120753,2.443114,3.335955,2.951765,3.589338,0.154104,0.184122,0.488746
336258,chrX,156022664,49447123,1834784,T,102,C,0.441176,0.555,28|23|1|62|23|0|0|65|0|0|0|14|0|16|19|10|11|24...,...,0.014497,0.082277,0.120753,2.430175,3.335955,3.069647,3.589338,0.157102,0.187574,0.489247
336261,chrX,156023203,49447469,1835130,T,52,A,0.942308,2.487,19|11|0|0|23|15|0|0|0|11|4|7|0|13|24|0|0|0|11-...,...,0.486635,0.105416,0.120753,2.895314,3.335955,3.564840,3.589338,0.184533,0.202075,0.475000


### find the CNV regions

### expand the regions according to leftCNV and rightCNV

In [367]:
df = std_df.query('Chr == "chr17"')

In [370]:
df['cnvL'] = (df['scoreL'] > 0.4).astype(int)
df['cnvR'] = (df['scoreR'] > 0.4).astype(int)
df[40:90]

<ipython-input-370-a12cce1e61ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cnvL'] = (df['scoreL'] > 0.4).astype(int)
<ipython-input-370-a12cce1e61ed>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cnvR'] = (df['scoreR'] > 0.4).astype(int)


,Chr,Start,FullExonPos,ExonPos,Ref,Depth,Alt,VAF,EBscore,PoN-Alt,...,stdR,sumL,sumR,dVAFL,dVAFR,scoreL,scoreR,scoreDiff,cnvL,cnvR
256380,chr17,3062684,38071381,115731,A,184,G,0.211957,0.923,36|48|68|0|63|21|0|24|0|0|0|11|20|0|14|46|0|0|...,...,0.257865,5.011788,10.427118,3.182760,8.179728,0.234076,0.398536,0.636870,0,0
256383,chr17,3062879,38071576,115926,G,538,C,0.241636,0.792,70|74|108|0|227|57|0|105|0|0|0|60|60|0|68|127|...,...,0.258499,5.011788,10.455113,3.182760,8.189949,0.234076,0.399356,0.637773,0,0
256384,chr17,3062897,38071594,115944,G,520,A,0.223077,0.784,69|74|103|0|217|57|0|101|0|0|0|55|52|0|67|127|...,...,0.261579,5.011788,10.650789,3.182760,8.841693,0.234076,0.405086,0.644086,0,1
256385,chr17,3062979,38071676,116026,G,521,A,0.253359,0.769,92|69|114|0|245|67|0|104|0|0|0|73|51|0|79|123|...,...,0.262673,5.011788,10.791580,3.182760,8.859459,0.234076,0.409209,0.648628,0,1
256387,chr17,3063014,38071711,116061,C,530,T,0.258491,0.776,86|74|109|0|237|71|0|107|0|0|0|61|49|0|68|98|0...,...,0.262049,5.011788,10.874973,3.182760,8.916858,0.234076,0.411651,0.651319,0,1
256420,chr17,3197533,38075163,119513,T,234,C,0.470085,0.686,79|142|163|0|183|0|0|236|163|0|65|0|0|121|68|0...,...,0.262897,5.011788,11.050689,3.182760,8.998918,0.234076,0.416796,0.656988,0,1
256430,chr17,3198284,38075863,120213,G,303,T,0.465347,0.794,93|119|135|0|182|0|0|257|150|0|58|0|0|186|87|0...,...,0.267095,5.011788,11.451049,3.182760,8.829198,0.234076,0.428520,0.669904,0,1
256432,chr17,3198296,38075875,120225,C,303,T,0.518152,1.335,100|97|153|0|0|79|0|0|0|0|0|81|89|0|84|216|0|0...,...,0.271971,5.011788,11.727498,3.182760,9.525355,0.234076,0.436615,0.678824,0,1
256434,chr17,3198397,38075976,120326,G,277,T,0.440433,0.716,88|94|83|1|109|0|0|203|126|0|42|0|0|115|71|0|6...,...,0.276298,5.011788,12.086539,3.182760,10.180193,0.234076,0.447129,0.690407,0,1
256440,chr17,3216379,38076767,121117,A,331,G,0.495468,1.454,86|93|111|0|0|81|0|0|0|0|0|85|84|0|92|167|0|0|...,...,0.281446,5.011788,12.377718,3.182760,10.830413,0.234076,0.455655,0.699801,0,1


### now do a rolling scan for more than x zeros and fill that up 
#### increases the 

In [355]:
df

,Chr,Start,FullExonPos,ExonPos,Ref,Depth,Alt,VAF,EBscore,PoN-Alt,...,stdL,stdR,sumL,sumR,dVAFL,dVAFR,scoreL,scoreR,scoreDiff,isCNV
86281,chr5,140417,13328861,150,T,220,C,0.486364,1.343,72|0|91|104|0|64|0|0|0|0|0|0|19|0|0|13|10|0|0-...,...,0.112810,0.112810,3.696555,3.696555,3.877363,3.877363,0.232084,0.244040,0.450115,0
86283,chr5,140521,13328965,254,G,269,A,0.498141,1.543,71|0|82|0|0|36|0|0|0|0|0|0|32|0|0|28|31|0|0-20...,...,0.112810,0.112805,3.696555,3.695788,3.877363,3.810539,0.232084,0.244015,0.450091,0
86285,chr5,140673,13329117,406,A,284,G,0.461268,1.352,60|0|70|55|0|82|0|0|0|0|0|0|36|0|0|39|26|0|0-5...,...,0.112810,0.112559,3.696555,3.638712,3.877363,3.843294,0.232084,0.242176,0.448309,0
86287,chr5,143082,13329311,600,G,334,A,0.431138,1.326,119|0|142|136|0|127|0|0|0|0|0|0|62|0|0|47|62|0...,...,0.112810,0.112578,3.696555,3.644477,3.877363,3.832389,0.232084,0.242362,0.448489,0
86289,chr5,143419,13329648,937,G,350,A,0.428571,1.272,103|0|124|96|0|151|0|0|0|2|0|0|43|0|0|46|31|0|...,...,0.112810,0.112560,3.696555,3.642760,3.877363,3.825081,0.232084,0.242307,0.448436,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97967,chr5,180630293,15520654,2191943,T,212,C,0.495283,1.589,44|0|7|0|0|0|69|0|0|0|0|0|0|40|0|0|0|0|0-36|0|...,...,0.142974,0.142917,4.431092,4.480911,5.389346,5.375890,0.304700,0.322663,0.473263,0
98039,chr5,180908671,15526802,2198091,G,277,A,0.498195,1.557,97|0|21|0|0|0|0|0|0|56|0|0|0|0|0|0|0|57|0-39|0...,...,0.143062,0.142917,4.441287,4.480911,5.388633,5.375890,0.304994,0.322640,0.472952,0
98044,chr5,180950018,15528221,2199510,C,214,A,0.504673,1.517,50|0|9|83|0|0|0|0|0|40|0|41|0|0|0|0|0|31|0-22|...,...,0.142929,0.142917,4.429385,4.480911,5.366536,5.375890,0.303939,0.321918,0.473278,0
98061,chr5,181045498,15530795,2202084,C,199,T,0.422111,0.506,59|0|6|122|65|98|146|66|0|0|62|32|0|0|54|31|35...,...,0.142905,0.142917,4.459655,4.480911,5.384904,5.375890,0.305448,0.322518,0.472388,0


In [365]:
std_df.to_csv(os.path.join(output_path, 'heteroSNP/01_A.snp'), sep='\t', index=False)

## OUTPUT

In [109]:
std_df.to_csv(os.path.join(output_path, 'heteroSNP/01_A.snp'), sep='\t', index=False)
cov_df.to_csv(os.path.join(output_path, 'covDif/01_A.snp'), sep='\t', index=False)

In [201]:
snp_df

,Chr,Start,FullExonPos,FullExonPos,ExonPos,Ref,Depth,Alt,VAF,EBscore,PoN-Alt
0,chr1,14930,836,836,836,A,64,G,0.046875,0.505,1|2|1|1|3|0|2|2|0|0|1|0|0|0|0|0|1|0|2-0|2|0|0|...
1,chr1,15820,991,991,991,g,13,T,0.846154,2.401,16|0|0|1|0|0|28|0|0|0|0|0|0|0|0|0|3|0|0-0|0|0|...
2,chr1,16949,1410,1410,1410,A,182,C,0.038462,0.006,10|27|7|23|12|4|8|3|2|0|0|1|4|1|1|0|3|1|4-0|53...
3,chr1,17365,1689,1689,1689,C,285,G,0.140351,0.666,26|0|0|0|17|20|0|0|0|16|6|13|3|14|9|1|0|0|11-1...
4,chr1,17375,1699,1699,1699,A,353,G,0.065156,1.526,15|0|1|0|0|0|0|0|0|0|0|0|3|3|0|0|0|0|0-0|0|0|0...
...,...,...,...,...,...,...,...,...,...,...,...
336263,chrX,156023260,49447526,49447526,1835187,G,25,C,0.880000,0.950,4|6|1|5|7|9|5|3|4|2|3|1|2|3|6|1|3|3|4-25|20|31...
336264,chrX,156023266,49447532,49447532,1835193,C,25,T,0.880000,0.930,3|6|1|3|5|7|3|3|2|2|2|1|2|3|5|0|3|3|3-25|16|29...
336265,chrX,156024203,49447762,49447762,1835423,G,53,A,0.132075,0.392,2|2|0|13|2|0|4|6|0|1|0|0|0|0|4|2|2|2|5-3|2|0|3...
336266,chrX,156024239,49447798,49447798,1835459,C,56,G,0.267857,0.203,15|9|5|18|8|3|5|8|6|6|0|8|1|5|10|9|4|5|5-6|3|3...


### combine SNP with local coverage info

In [280]:
std_df

,Chr,Start,FullExonPos,ExonPos,Ref,Depth,Alt,VAF,EBscore,PoN-Alt,offCenter,stdL,stdR,sumL,sumR,scoreL,scoreR,scoreDiff,scoreSum
3,chr1,17365,1689,1689,C,285,G,0.140351,0.666,26|0|0|0|17|20|0|0|0|16|6|13|3|14|9|1|0|0|11-1...,0.298182,NaN,0.144470,NaN,4.512223,NaN,0.406860,NaN,NaN
4,chr1,17375,1699,1699,A,353,G,0.065156,1.526,15|0|1|0|0|0|0|0|0|0|0|0|3|3|0|0|0|0|0-0|0|0|0...,0.373377,NaN,0.138932,NaN,4.262098,NaN,0.369576,NaN,NaN
5,chr1,17379,1703,1703,G,363,A,0.093664,1.578,20|0|22|0|0|0|0|0|0|0|0|0|0|0|0|0|3|0|0-3|0|5|...,0.344869,NaN,0.129318,NaN,3.937145,NaN,0.317773,NaN,NaN
8,chr1,17614,1938,1938,G,216,A,0.171296,1.016,9|1|0|0|3|3|0|0|0|3|1|2|0|0|4|0|0|0|3-17|0|0|0...,0.267237,NaN,0.119883,NaN,3.610519,NaN,0.270150,NaN,NaN
12,chr1,69968,3629,3629,A,32,G,0.093750,1.149,0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0-0|0|0|0|...,0.344783,NaN,0.113784,NaN,3.391969,NaN,0.240887,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336251,chrX,156010159,49446143,1833804,A,59,G,0.576271,0.927,13|17|16|0|0|0|0|20|0|0|0|21|0|6|0|0|0|19|2-4|...,0.137738,0.080648,NaN,2.891634,NaN,0.267792,NaN,NaN,NaN
336252,chrX,156010162,49446146,1833807,A,58,G,0.586207,1.072,12|18|16|0|0|0|0|21|0|0|0|20|0|6|0|0|0|19|0-5|...,0.147674,0.082125,NaN,3.021882,NaN,0.284979,NaN,NaN,NaN
336258,chrX,156022664,49447123,1834784,T,102,C,0.441176,0.555,28|23|1|62|23|0|0|65|0|0|0|14|0|16|19|10|11|24...,0.002643,0.082277,NaN,2.979950,NaN,0.281545,NaN,NaN,NaN
336261,chrX,156023203,49447469,1835130,T,52,A,0.942308,2.487,19|11|0|0|23|15|0|0|0|11|4|7|0|13|24|0|0|0|11-...,0.503775,0.105416,NaN,3.445089,NaN,0.417029,NaN,NaN,NaN


In [281]:
cov_df

,Chr,Pos,FullExonPos,ExonPos,Coverage,PONmeanCov,PONmedianCov,PONstd,log2ratio,chromAccum
0,chr1,16750,1211,1211,73.635666,45.918297,45.918297,18.882631,0.681336,0
1,chr1,16950,1411,1411,71.948120,31.275524,30.173918,13.179594,1.201923,0
2,chr1,17000,1461,1461,59.263895,21.266547,19.518825,10.704707,1.478568,0
3,chr1,17300,1624,1624,65.159679,19.347489,19.595966,6.948574,1.751833,0
4,chr1,17350,1674,1674,106.942382,38.241357,38.241357,11.840340,1.483628,0
...,...,...,...,...,...,...,...,...,...,...
690681,chrX,156024250,49447809,1835470,24.707203,22.130863,22.130863,9.875716,0.158872,47612339
690682,chrX,156025350,49448172,1835833,25.119488,22.331925,20.457632,12.148273,0.169699,47612339
690683,chrX,156027300,49448383,1836044,23.984212,28.676397,30.138524,11.504929,-0.257779,47612339
690684,chrX,156027350,49448433,1836094,22.850927,29.436317,29.042386,13.134193,-0.365345,47612339
